In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.densenet import DenseNet121,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortex"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = DenseNet121(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 1, 1, 1024)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 1024)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   44.9s


[CV] ........................... C=0.01, score=0.915039, total=  45.3s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.938553, total=  46.5s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.934263, total=  46.4s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.906496, total=  47.6s
[CV] ........................... C=0.01, score=0.932740, total=  47.5s
[CV] C=1.0 ...........................................................
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.913725, total=  49.5s
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.931548, total=  49.5s
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.3min


[CV] ............................ C=1.0, score=0.905512, total= 2.3min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.935580, total= 2.3min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.912402, total= 2.3min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.937376, total= 2.3min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.939664, total= 2.3min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.912745, total= 2.3min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.940239, total= 2.3min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.923679, total= 3.5min
[CV] ................... C=1000000000.0, score=0.900000, total= 3.6min
[CV] ................... C=1000000000.0, score=0.902559, total= 3.7min
[CV] ................... C=1000000000.0, score=0.919881, total= 3.7min
[CV] ................... C=1000000000.0, score=0.923687, total= 3.6min
[CV] .................. C=10000000000.0, score=0.899606, total= 3.3min
[CV] .................. C=10000000000.0, score=0.907227, total= 3.5min
[CV] .................. C=10000000000.0, score=0.927593, total= 3.5min
[CV] ................... C=1000000000.0, score=0.933400, total= 3.7min
[CV] .................. C=10000000000.0, score=0.912745, total= 3.6min
[CV] ................... C=1000000000.0, score=0.924303, total= 3.7min
[CV] ................... C=1000000000.0, score=0.921627, total= 3.8min
[CV] .................. C=10000000000.0, score=0.915354, total= 3.5min
[CV] ................... C=1000000000.0, score=0.915354, total= 4.0min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 13.9min remaining:   33.4s


[CV] .................. C=10000000000.0, score=0.918892, total= 3.5min
[CV] .................. C=10000000000.0, score=0.921627, total= 3.1min
[CV] .................. C=10000000000.0, score=0.928642, total= 3.3min
[CV] .................. C=10000000000.0, score=0.937376, total= 3.5min
[CV] .................. C=10000000000.0, score=0.923307, total= 3.2min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 14.9min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([ 47.01346128, 138.93001373, 139.39786534, 138.82310314,
        143.70024719, 142.12681186, 143.11039112, 142.45657988,
        145.36429641, 159.97153277, 189.98423281, 220.36171966,
        204.31722422]),
 'mean_score_time': array([0.0245157 , 0.032498  , 0.02217031, 0.02136226, 0.02038147,
        0.01978257, 0.01689055, 0.01664295, 0.02403045, 0.02889931,
        0.03312421, 0.02219741, 0.02483704]),
 'mean_test_score': array([0.92719021, 0.92936069, 0.92196133, 0.91880426, 0.91466062,
        0.91436464, 0.91377269, 0.91791634, 0.91732439, 0.91742305,
        0.91692976, 0.91712707, 0.9191989 ]),
 'mean_train_score': array([0.96229109, 0.9876565 , 0.99381722, 0.99354375, 0.98389609,
        0.98858478, 0.98749461, 0.99238127, 0.99362015, 0.99305074,
        0.99159306, 0.99134917, 0.99186643]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.92719, std: 0.01060, params: {'C': 0.01},
 mean: 0.92936, std: 0.01016, params: {'C': 0.1},
 mean: 0.92196, std: 0.01013, params: {'C': 1.0},
 mean: 0.91880, std: 0.01008, params: {'C': 10.0},
 mean: 0.91466, std: 0.01468, params: {'C': 100.0},
 mean: 0.91436, std: 0.01328, params: {'C': 1000.0},
 mean: 0.91377, std: 0.01163, params: {'C': 10000.0},
 mean: 0.91792, std: 0.00967, params: {'C': 100000.0},
 mean: 0.91732, std: 0.01026, params: {'C': 1000000.0},
 mean: 0.91742, std: 0.00961, params: {'C': 10000000.0},
 mean: 0.91693, std: 0.00924, params: {'C': 100000000.0},
 mean: 0.91713, std: 0.01018, params: {'C': 1000000000.0},
 mean: 0.91920, std: 0.01047, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.1} - score: 0.9294
